# AMPLab 2025 Processing large datasets and Collaborative Filtering

Install dependencies in a separate cell, so that if people run it in jupyter with dependencies already present, they can choose to omit it.

We also install the `zstd` commandline tool to process our data files

In [ ]:
%pip install implicit numpy scipy h5py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 20.0 MB/s eta 0:00:00


In [ ]:
!apt install -y zstd

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  zstd
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 603 kB of archives.
After this operation, 1,695 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 zstd amd64 1.4.8+dfsg-3build1 [603 kB]
Fetched 603 kB in 0s (2,430 kB/s)
Selecting previously unselected package zstd.
(Reading database ... 124788 files and directories currently installed.)
Preparing to unpack .../zstd_1.4.8+dfsg-3build1_amd64.deb ...
Unpacking zstd (1.4.8+dfsg-3build1) ...
Setting up zstd (1.4.8+dfsg-3build1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import collections
import csv
import json
import os
import shutil
import glob

import pandas as pd

### Memory management
Here's a quick way of seeing what variables are using up a lot of memory.

```py
import sys
global_keys = list(globals().keys())
for k in global_keys:
  if not k.startswith("_"):
    size = sys.getsizeof(globals()[k])
    if size > 10000:
      print(f"{k}: {round(size / 1024 / 1024, 2)}MB")
```

You can delete any variable by running

```py
thevariable = None
```

which will cause python to garbage-collect it and free up the memory. It's worth doing this from time to time in colab if you are at risk of running out of memory in the runtime.


# Part 1: Data processing

## Step 1: Reading initial data files

In [ ]:
# Where the original data files are - use a google drive shortcut so that you don't need to copy them
data_root = "/content/drive/MyDrive/amplab/2025-materials"
# Local data directory for items that you don't want to lose
working_root = "/content/drive/MyDrive/amplab-working"
# Temporary location that you can use while the colab is running
scratch_root = "/content/amplab-scratch"
os.makedirs(working_root, exist_ok=True)
os.makedirs(scratch_root, exist_ok=True)

Double-check the data directory to see what's there

In [ ]:
!ls -lh {data_root}

total 14G
-rw------- 1 root root 618M Jan 22 14:41 10.listens.zst
-rw------- 1 root root 619M Jan 22 14:41 11.listens.zst
-rw------- 1 root root 636M Jan 22 14:41 12.listens.zst
-rw------- 1 root root 756M Jan 22 14:40 1.listens.zst
-rw------- 1 root root 711M Jan 22 14:42 2.listens.zst
-rw------- 1 root root 738M Jan 22 14:42 3.listens.zst
-rw------- 1 root root 732M Jan 22 14:43 4.listens.zst
-rw------- 1 root root 708M Jan 22 14:43 5.listens.zst
-rw------- 1 root root 627M Jan 22 14:44 6.listens.zst
-rw------- 1 root root 645M Jan 22 14:44 7.listens.zst
-rw------- 1 root root 597M Jan 22 14:44 8.listens.zst
-rw------- 1 root root 590M Jan 22 14:45 9.listens.zst
-rw------- 1 root root 1.5G Jan 22 14:39 canonical_musicbrainz_data.csv.zst
-rw------- 1 root root 237M Jan 27 12:25 canonical_recording_redirect.csv.zst
-rw------- 1 root root 1.9K Jan 29 15:32 listenbrainz_model.py
-rw------- 1 root root 3.4G Jan 23 15:44 listenbrainz_msid_mapping.csv.zst
-rw------- 1 root root 106M Jan 22 

The data files are compressed using the zstandard format. Colab has a "local disk" that we can use temporarily. This is not saved when the notebook notebook is closed, but is faster than reading directly from google drive. We can copy a file to our drive and uncompress it to inspect it.

We know that a data file is in "jsonlines" format, where each line of the file is a json document. We can inspect the first line to see the structure of this document, and see that we want to read the `user_id` and `recording_msid` fields.

In [ ]:
scratch_listen_1_compressed = shutil.copy2(os.path.join(data_root, "1.listens.zst"), scratch_root)
!unzstd {scratch_listen_1_compressed}
!head -2 {os.path.join(scratch_root, "1.listens")}

/content/amplab-scratch/1.listens.zst: 5199323140 bytes 
{"user_id":24848,"user_name":"Tarocus","timestamp":1704067200,"track_metadata":{"track_name":"Kapitel 13 - Der Krieg der Knöpfe","artist_name":"...mit Pauken und Trompeten, Louis Pergaud, Laura Maire, Jens Wawrczeck, Stefan Kaminski","release_name":"Der Krieg der Knöpfe","additional_info":{"isrc":"DEXO42360570","discnumber":1,"origin_url":"https://open.spotify.com/track/33nDWmcHOUunB26hLRN1oz","spotify_id":"https://open.spotify.com/track/33nDWmcHOUunB26hLRN1oz","duration_ms":210848,"tracknumber":13,"artist_names":["...mit Pauken und Trompeten","Louis Pergaud","Laura Maire","Jens Wawrczeck","Stefan Kaminski"],"music_service":"spotify.com","spotify_album_id":"https://open.spotify.com/album/0OSmDFaDD2tzOFCyECMI7x","submission_client":"listenbrainz","spotify_artist_ids":["https://open.spotify.com/artist/7avdi56utuo9mPydRE5gUL","https://open.spotify.com/artist/4en4B4bJyaig2IQoRF7Cwo","https://open.spotify.com/artist/6UoPOljQFeWdRBinxz

We know that a data file is in "jsonlines" format, where each line of the file is a json document. We can inspect the first line to see the structure of this document, and see that we want to read the `user_id` and `recording_msid` fields.

In [ ]:
with open(os.path.join(scratch_root, "1.listens")) as fp:
  line = fp.readline()
  line_json = json.loads(line)
  print(json.dumps(line_json, indent=4))

{
    "user_id": 24848,
    "user_name": "Tarocus",
    "timestamp": 1704067200,
    "track_metadata": {
        "track_name": "Kapitel 13 - Der Krieg der Kn\u00f6pfe",
        "artist_name": "...mit Pauken und Trompeten, Louis Pergaud, Laura Maire, Jens Wawrczeck, Stefan Kaminski",
        "release_name": "Der Krieg der Kn\u00f6pfe",
        "additional_info": {
            "isrc": "DEXO42360570",
            "discnumber": 1,
            "origin_url": "https://open.spotify.com/track/33nDWmcHOUunB26hLRN1oz",
            "spotify_id": "https://open.spotify.com/track/33nDWmcHOUunB26hLRN1oz",
            "duration_ms": 210848,
            "tracknumber": 13,
            "artist_names": [
                "...mit Pauken und Trompeten",
                "Louis Pergaud",
                "Laura Maire",
                "Jens Wawrczeck",
                "Stefan Kaminski"
            ],
            "music_service": "spotify.com",
            "spotify_album_id": "https://open.spotify.com/album/0OSmD

We can read a file one line at a time, parse the json, and save the user id and recording msid.
Note that if you do this for all files then the `data` variable may cause you to run out of memory. Consider this in your work

In [ ]:
%%time
data = []
with open(os.path.join(scratch_root, "1.listens")) as fp:
  for line in fp:
    j = json.loads(line)
    user_id = j['user_id']
    recording_msid = j['recording_msid']
    data.append([user_id, recording_msid])

CPU times: user 2min 13s, sys: 4.75 s, total: 2min 18s
Wall time: 2min 45s


In [ ]:
# Save the data that we extracted to a single file, in case the colab runtime quits
# The arguments to `open` are best practices when reading CSV files in python, although I don't always use them
# https://docs.python.org/3/library/csv.html#csv.reader
with open(os.path.join(working_root, "userid-msid.csv"), "w", newline='', encoding='utf-8') as fp:
    w = csv.writer(fp)
    w.writerows(data)

In [ ]:
# Load the data from the previous cell again, if we stopped at this point
data = []
with open(os.path.join(working_root, "userid-msid.csv"), "r", newline='', encoding='utf-8') as fp:
    r = csv.reader(fp)
    for line in r:
        data.append(line)

In [ ]:
# Double-check the total number of lines of data we have
!wc -l {os.path.join(working_root, "userid-msid.csv")}

8839350 /content/drive/MyDrive/amplab-working/userid-msid.csv


## Step 2: Mapping MSID to MBID

In [ ]:
listenbrainz_msid_mapping_fn = shutil.copy2(os.path.join(data_root, "listenbrainz_msid_mapping.csv.zst"), scratch_root)
!unzstd {listenbrainz_msid_mapping_fn}

/content/amplab-scratch/listenbrainz_msid_mapping.csv.zst: 8215704700 bytes 


Try and load the mapping CSV into memory. Ideally I want to have a dictionary such as:

```py
mapping = {
  "msid1": "mbid1",
  "msid2": "mbid"
}
```

which would allow us to look up an msid in the `data` list above, with just

```py
mbid = mapping[data[0][1]]
```

Unfortunately, this CSV file is too large for colab, and it will run out of memory before loading the entire file.

However, we can see that the size of the mapping file (111 million rows) is much larger than the number of unique MSIDs that we have in the data file

In [ ]:
# On colab this cell will run out of memory
mapping = {}
with open(os.path.join(scratch_root, "listenbrainz_msid_mapping.csv")) as fp:
    r = csv.reader(fp)
    # omit header
    next(r)
    for line in r:
        mapping[line[0]] = line[1]

In [ ]:
!head -40 {os.path.join(scratch_root, "listenbrainz_msid_mapping.csv")}

recording_msid,recording_mbid,match_type
13ca445f-c0dd-4f64-8726-7da78a3821aa,,no_match
54a40ef8-6bfe-4803-b74a-7b93885c2f01,,no_match
21c07966-97c6-4e02-a575-e1b2fadf0d34,,no_match
3b438b25-b9ad-480f-b248-bd06281919e0,,no_match
26d8cc2b-c249-4b70-9f16-4eea1419303c,,no_match
bf3fe1e0-7ae6-4e57-a38e-6ac7d42d33c3,,no_match
937d9d97-ba8b-4ff4-9af7-1b4a485945c1,,no_match
0e9230f2-1d9d-47ab-a44f-dbc788cfbdf1,,no_match
2c9111a7-96e6-4083-8b45-78cadb11796e,,no_match
012868b2-f6d7-40e6-80a4-d89d077e3d9e,,no_match
03feccff-3632-4acf-974b-b787ff7e9bbf,,no_match
364902a8-1067-4804-b8be-d6b801ee4179,,no_match
13a44caa-8109-46ef-ac25-0e89785bdd18,,no_match
273849c3-b1d7-4656-bf67-748c6cac2179,,no_match
da3aea98-c857-4eb7-b4d6-cce8aa0318d7,,no_match
c514427c-89e1-4746-86bb-49f5d5c8e04a,,no_match
67c2a304-1ff5-44ea-a076-9d797ba15c28,,no_match
127b9ad4-20d6-49e0-ab82-5aad8d27b5e1,,no_match
703adaef-f000-4e6c-bd96-6c9c299af991,,no_match
6848ccaf-4942-4fcf-bde3-fbe4e86ede21,,no_match
562560cc-a8a0-4857-

In [ ]:
mapping_file = os.path.join(scratch_root, "listenbrainz_msid_mapping.csv")
!wc -l {mapping_file}

111339170 /content/amplab-scratch/listenbrainz_msid_mapping.csv


In [ ]:
unique_msids = set(line[1] for line in data)
print(f"Unique msids: {len(unique_msids)}")
# Note that this is even smaller than the 8.8m lines in the file `userid-msid.csv`, because we're removing duplicates.

Unique msids: 3013073


Therefore, we can read the mapping file one line at a time and make a new small mapping file containing only the MSID values that we need.

**Trick:** This is one of the tricks to work with large amounts of data without loading it into memory. You can read the file one line at a time (python does this if you iterate over a csv reader such as `for line in r`), or you could use `fp.readline`. You could also do a variant of this by reading a data frame in pandas in chunks (e.g. 100,000 lines at a time), and processing each chunk before moving on to the next one.

I decided to read all lines in the mapping, but you could also decide to use only exact match or high quality results.
If you look at the mapping file you can see that the 3rd column contains a "match_type" field representing the quality of the match. This can be "exact_match", "high_quality", "med_quality", "low_quality", or "no_match"

In [ ]:
!head -1000 {os.path.join(scratch_root, "listenbrainz_msid_mapping.csv")} | cut -d, -f3 | sort -u

exact_match
high_quality
low_quality
match_type
med_quality
no_match


In [ ]:
# Here we don't even need to store the entire contents of these files.
# Instead we just read the first file 1 line and a time and write a new line directly to the 2nd file if needed.
with open(os.path.join(scratch_root, "listenbrainz_msid_mapping.csv")) as r_fp, open(os.path.join(scratch_root, "small_msid_mapping.csv"), "w") as w_fp:
    r = csv.reader(r_fp)
    w = csv.writer(w_fp)
    header = next(r)
    w.writerow(header)
    for line in r:
      if line[0] in unique_msids and line[2] != "no_match":
        w.writerow(line)

In [ ]:
highquality_mapping = {}
with open("listenbrainz_msid_mapping.csv") as fp:
    r = csv.reader(fp)
    next(r)
    for line in r:
        if line[2] == "exact_match" or line[2] == "high_quality":
            highquality_mapping[line[0]] = line[1]

Now that we have a small mapping file, we know that it will fit in memory. Until now, `data` has been in memory (a list of userid/msid pairs).

We can free up the `data` variable from memory, load the mapping, and then process the "userid-msid.csv" file, one line at a time, immediately writing out a new file with the relevant MBID.

In [ ]:
data = None # throw away our `data` list, this will free memory.
smallmapping = {}
with open(os.path.join(scratch_root, "small_msid_mapping.csv")) as fp:
  reader = csv.reader(fp)
  next(reader)
  for line in reader:
    smallmapping[line[0]] = line[1]

## Continue here...


The remainer of your code to generate the final data file for the collaborative filtering model can go here.

Remember to check the number of items in your dataset after applying each data conversion to ensure that you're correctly processing the dataset.

# Part 2: Collaborative filtering model

This part of the assignment is based on the [implicit tutorial](https://benfred.github.io/implicit/tutorial_lastfm.html) and builds a collaborative filtering model. We reuse as much code as possible from this library.

We can find this source code on the [project's github page](https://github.com/benfred/implicit), specifically https://github.com/benfred/implicit/blob/main/implicit/datasets/lastfm.py has code which builds the model based on the last.fm dataset.

We've extracted the relevant code from this file and modified it for this task. It's in `listenbrainz_model.py`

## Step 6: Generate matrix

In [ ]:
# Add data_root to the python path so that we can load the provided file.
# You might want to move this file somewhere else and edit it, update the path as necessary
import sys
sys.path.append(data_root)

In [ ]:
import listenbrainz_model as lb

In [ ]:
matrix_artists, matrix_users, plays = lb.load_data_matrix(os.path.join(working_root, "userid-artist-counts.csv"))

## Step 7: Build model

In [ ]:
# On colab, this only takes about 2-3 minutes.
model = lb.build_model(plays)

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

We used Artist MBIDs in our data file in order to be unique, so we need one final mapping which allows us to go from an artist MBID to a name.

In [ ]:
artist_map = lb.get_artist_map(os.path.join(data_root, "musicbrainz_artist.csv"))
def format_artist(artist_id):
    return f"""<a href="https://musicbrainz.org/artist/{artist_id}">{artist_map.get(artist_id, "unknown")}</a> ({artist_id})"""

In [ ]:
# get related items for The Clash (MBID = 8f92558c-2baa-4758-8c38-615519e9deda)
ids, scores = model.similar_items(lb.artist_index(matrix_artists, "8f92558c-2baa-4758-8c38-615519e9deda"), N=20)

# use pandas for nicer formatting
df = pd.DataFrame({"artist": [format_artist(a) for a in matrix_artists[ids]], "score": scores})

In [ ]:
from IPython.display import HTML

HTML(df.to_html(escape=False))